___


# <font color= #8A0829> Proyecto Taller de Modelado de Datos </font>

<Strong> Alumnos: </Strong>
- Marcela Quintero Pérez
- Santiago Ayon Sanchez
- Gael Rendon Mendoza

<Strong> Año </Strong>: 2025

<Strong> Email: </Strong>  
- <font color="blue"> is717644@iteso.mx </font>
- <font color="blue"> santiago.ayon@iteso.mx </font>
- <font color="blue"> gael.rendon@iteso.mx </font>

___

## INSTRUCCIONES GENERALES PARA AMBOS PROBLEMAS

1.1 **Visualización, exploración, preparación y análisis de datos:** 
- Extracción de información estadística básica en los datos analizados.
- En esta fase las características de las bases de datos deben ser mencionadas (datos nulos, datos sesgados, datos atípicos, distribución de datos, etc.)
- Limpieza de datos
- En esta fase de debe mencionar y mostrar el proceso y decisiones tomadas para procesar y limpiar estos datos. 
- Se deben incluir gráficos interesantes que muestren el comportamiento de los datos y cómo se relacionan, así como acompañar los gráficos con una explicación de lo que se observa en el gráfico.
- Después de haber hecho el análisis de tus datos, puedes irte dando una idea de cuál modelo es el mejor para tu dataset. 
- Explica dadas las caracterícas que observaste sobre los datos que tienes, cuál sería el mejor modelo a aplicar en tu dataset. 

## Librerías

In [10]:
# Librerías
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.colors as pc
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
from sklearn.discriminant_analysis import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score, learning_curve, train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from ucimlrepo import fetch_ucirepo 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from collections import Counter

# Ignorar warnings
import warnings
warnings.filterwarnings("ignore")

colors = pc.qualitative.Plotly + pc.qualitative.D3

sns.set_theme(style="whitegrid")

seed = 123

In [11]:
# pip install ucimlrepo

## Clasificación

###  <font color= #ffffffff> Estimation of Obesity Levels Based On Eating Habits and Physical Condition </font>
url: https://archive.ics.uci.edu/dataset/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition

Este conjunto de datos fue recopilado para estimar los niveles de obesidad en individuos de México, Perú y Colombia, basándose en sus hábitos alimenticios y condición física. El 77% de los datos fueron generados sintéticamente utilizando la herramienta Weka y la técnica SMOTE, y el 23% de los datos restantes fueron recolectados directamente de usuarios a través de una plataforma web.


### Descripción de las variables del dataset

- **Gender:** Género del individuo. Valores posibles: `Male` (Masculino), `Female` (Femenino).
- **Age:** Edad del individuo en años.
- **Height:** Altura del individuo en metros.
- **Weight** Peso del individuo en kilogramos.
- **family_history_with_overweight:** Indica si hay antecedentes familiares de sobrepeso. Valores: `yes` (Sí), `no` (No).
- **FAVC:** Frecuencia de consumo de alimentos con alto contenido calórico. Valores: `yes` (Sí), `no` (No).
- **FCVC:** Frecuencia de consumo de vegetales. Valores discretos: 1 (Bajo) a 3 (Alto).
- **NCP:** Número de comidas principales diarias. Valores discretos: 1 a 4.
- **CAEC:** Consumo de alimentos entre comidas. Valores: `no`, `Sometimes` (A veces), `Frequently` (Frecuentemente), `Always` (Siempre).
- **CH2O:** Consumo de agua diaria (litros). Valores discretos: 1 a 3.
- **FAF:** Frecuencia de actividad física. Valores discretos: 0 a 3.
- **TUE:** Tiempo de exposición a pantallas (televisión, computadora, videojuegos). Valores discretos: 0 a 3.
- **CALC:** Consumo de alcohol. Valores: `yes` (Sí), `no` (No).
- **SMOKE:** Consumo de tabaco. Valores: `yes` (Sí), `no` (No).
- **MTRANS:** Medio de transporte habitual. Valores: `Public_Transportation` (Transporte público), `Automobile` (Automóvil), `Walking` (Caminando), `Motorbike` (Motocicleta), `Bike` (Bicicleta).
- **NObeyesdad (target):** Nivel de obesidad del individuo (variable objetivo). 
    Valores:  
  - `Insufficient_Weight` (Peso insuficiente)  
  - `Normal_Weight` (Peso normal)  
  - `Overweight_Level_I` (Sobrepeso Nivel I)  
  - `Overweight_Level_II` (Sobrepeso Nivel II)  
  - `Obesity_Type_I` (Obesidad Tipo I)  
  - `Obesity_Type_II` (Obesidad Tipo II)  
  - `Obesity_Type_III` (Obesidad Tipo III)

In [12]:
# fetch dataset 
estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition = fetch_ucirepo(id=544) 
  
# data (as pandas dataframes) 
X = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.features 
y = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.targets 
  
# metadata 
# print(estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.metadata) 
  
# variables 
# print(estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.variables) 

### EDA

In [13]:
# Combinar en un solo DataFrame
df_clasificacion = pd.concat([X, y], axis=1)
df_clasificacion_copy = df_clasificacion.copy()

df_clasificacion.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


El conjunto de datos cuenta con 17 variables y 2111 observaciones, los cuales están etiquetados con la variable objetivo NObesity que es el nivel de obesidad.

In [14]:
print("Dimensiones del dataset:", df_clasificacion.shape)

print("\nInfo del dataset:")
print(df_clasificacion.info())

Dimensiones del dataset: (2111, 17)

Info del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non

El dataset no presenta valores nulos en ninguna columna, por lo que no es necesario realizar imputación de datos. Sin embargo, sí contiene registros duplicados, los cuales se procederán a eliminar.

In [15]:
# Revisar valores nulos y duplicados
print("\nValores nulos por columna:")
print(df_clasificacion.isnull().sum())

print("\nNúmero de registros duplicados:", df_clasificacion.duplicated().sum())
df = df_clasificacion.drop_duplicates()


Valores nulos por columna:
Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

Número de registros duplicados: 24


In [16]:
print("\nEstadísticas descriptivas:")
print(df_clasificacion.describe().T)


Estadísticas descriptivas:
         count       mean        std    min        25%        50%         75%  \
Age     2111.0  24.312600   6.345968  14.00  19.947192  22.777890   26.000000   
Height  2111.0   1.701677   0.093305   1.45   1.630000   1.700499    1.768464   
Weight  2111.0  86.586058  26.191172  39.00  65.473343  83.000000  107.430682   
FCVC    2111.0   2.419043   0.533927   1.00   2.000000   2.385502    3.000000   
NCP     2111.0   2.685628   0.778039   1.00   2.658738   3.000000    3.000000   
CH2O    2111.0   2.008011   0.612953   1.00   1.584812   2.000000    2.477420   
FAF     2111.0   1.010298   0.850592   0.00   0.124505   1.000000    1.666678   
TUE     2111.0   0.657866   0.608927   0.00   0.000000   0.625350    1.000000   

           max  
Age      61.00  
Height    1.98  
Weight  173.00  
FCVC      3.00  
NCP       4.00  
CH2O      3.00  
FAF       3.00  
TUE       2.00  


Las columnas `family_history_with_overweight`, `FAVC`, `SMOKE` y `SCC` contienen valores categóricos binarios representados como `'yes'` y `'no'`. Para facilitar el análisis y la modelación, estos valores se codifican numéricamente, asignando 1 a `'yes'` y 0 a `'no'`.

In [17]:
bin = ['family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']

for col in bin:
    df_clasificacion[col] = df_clasificacion[col].map({'no': 0, 'yes': 1})

In [18]:
train_df_clas, test_df_clas = train_test_split(df_clasificacion, test_size=0.2, random_state=42)

print("Train shape:", train_df_clas.shape)
print("Test shape:", test_df_clas.shape)

Train shape: (1688, 17)
Test shape: (423, 17)


In [19]:
categorical_cols = df_clasificacion.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df_clasificacion.select_dtypes(include=['float64', 'int64']).columns.tolist()

categorical_cols.remove('NObeyesdad')

In [20]:
# print("Categorical variables:", categorical_cols)
# print("Numerical variables:", numerical_cols)

Para las variables numéricas, se procede a analizar su comportamiento respecto al target `NObeyesdad`:

In [21]:
for i, var in enumerate(numerical_cols):
    color = colors[i]

    fig = make_subplots(rows=1, cols=2, subplot_titles=(f"{var} vs NObeyesdad", f"Distribución de {var}"))

    scatter = go.Scatter(x=train_df_clas[var], y=train_df_clas["NObeyesdad"], mode="markers", marker=dict(color=color, opacity=0.6), name=f"{var} vs NObeyesdad")
    fig.add_trace(scatter, row=1, col=1)

    hist = go.Histogram(x=train_df_clas[var], marker_color=color, name=f"Distribución de {var}")
    fig.add_trace(hist, row=1, col=2)

    fig.update_layout(title_text=f"Análisis de {var}", title_font=dict(size=20, family="Arial", color="black"), title_x=0.5, showlegend=False)

    fig.show()

En los gráficos anteriores se pudo observar lo siguiente:

- **Age:**

    - La mayoría de los registros corresponden a personas jóvenes, con una mediana de 22.8 años y un rango de 14 a 61 años.
    - El 50% central de los datos se encuentra entre aproximadamente 20 y 26 años, lo que indica que la distribución está concentrada en edades jóvenes.

- **Height**:

    - La altura varía entre 1.45 y 1.98 m, con una mediana de 1.70 m.
    - Los datos están bastante concentrados, sin valores extremos, mostrando una distribución relativamente uniforme.

- **Weight**:

    - El peso varía ampliamente, desde 39 hasta 173 kg, con una mediana de 83 kg.
    - El rango intercuartílico indica que la mayoría de los registros se encuentra entre 65 y 107 kg, pero existen valores extremos que podrían considerarse outliers.

`Insight: Recomendable normalizar o estandarizar esta variable para análisis que dependan de la escala, y revisar valores atípicos antes de modelar.`

- **family_history_with_overweight, FAVC, SMOKE, SCC:**

    - family_history_with_overweight y FAVC presentan una alta concentración en 1 (sí), mientras que SMOKE y SCC tienen muy pocos casos positivos (1).

`Insight: Las variables binarias con una gran concentración en un solo valor pueden no aportar suficiente información en modelos de clasificación; se podrían considerar técnicas de balanceo o ponderación, o bien considerar eliminarlas.`

- **FCVC, NCP, CH2O, FAF, TUE:**

    - Estas variables muestran rangos limitados (1–4 o 1–3) y medianas cercanas al valor central, indicando que la mayoría de los registros se agrupan alrededor de ciertos valores.

`Insight: Pueden tratarse como variables ordinales o continuas según el modelo que se vaya a utilizar para clasificar.`

El dataset contiene variables con suficiente información para construir un modelo de clasificación, destacando `Height`, `Weight`, `Age` y `family_history_with_overweight` como las más relevantes. Las demás variables pueden ser complementarias, especialmente si se consideran hábitos y comportamientos, pero algunas requieren manejo del desbalance para ser útiles en predicciones.

Ahora se procede a analizar el comportamiento de las variables categóricas respecto al target `NObeyesdad`:

In [22]:
for i, col in enumerate(categorical_cols):
    color = colors[i]

    counts = train_df_clas[col].value_counts().reset_index()
    counts.columns = [col, "count"]

    fig = make_subplots(rows=1, cols=2, subplot_titles=(f"{col} vs NObeyesdad", f"Frecuencia de {col}"))

    fig_box = px.box(train_df_clas, x=col, y="NObeyesdad", points="all", color_discrete_sequence=[color])
    for trace in fig_box.data: fig.add_trace(trace, row=1, col=1)

    fig_count = px.bar(counts, x=col, y="count", text="count", color_discrete_sequence=[color])
    for trace in fig_count.data:
        fig.add_trace(trace, row=1, col=2)

    fig.update_layout(title_text=f"Análisis de {col}", title_font=dict(size=20, family="Arial", color="black"), title_x=0.5, showlegend=False)

    fig.show()

En los gráficos anteriores se pudo observar lo siguiente:

- **Gender:**

    - La distribución entre hombres (50.6%) y mujeres (49.4%) es prácticamente equilibrada.

- **CAEC:**

    - La mayoría de los registros corresponden a "Sometimes" (83.6%), mientras que "Frequently" (11.5%), "Always" (2.5%) y "no" (2.4%) son mucho menos frecuentes.

`Insight: Existe un desbalance importante entre categorías; podría considerarse agrupar "Frequently" y "Always" en una sola categoría para análisis o modelado.`

- **CALC:**

    - La categoría "Sometimes" concentra 66.4% de los registros, "no" 30.3%, mientras que "Frequently" (3.3%) y "Always" (0.05%) son muy escasas.

`Insight: Similar a CAEC, las categorías poco frecuentes podrían agruparse para reducir desbalance.`

- **MTRANS:**

    - La mayoría utiliza transporte público (74.8%) o automóvil (21.6%). Otras categorías (Walking, Motorbike, Bike) son muy minoritarias.

`Insight: Podría agruparse el grupo minoritario como "Otros" para simplificar el análisis y evitar desbalance.` 

Se pudo observar que de este grupo de variables, `Gender` es la única balanceada y que se pudiera considerar directamente útil para el análisis y modelado.

Las variables relacionadas con hábitos (CAEC, CALC, MTRANS) presentan desbalance fuerte en algunas categorías; en general es recomendable agrupar las categorías menos frecuentes para simplificar el análisis y mejorar la calidad del modelo en un futuro.

En general, estas variables categóricas aportan información valiosa sobre hábitos y características de los individuos, que complementan a las variables numéricas en la clasificación de sobrepeso y obesidad.

### Preprocesamiento

In [23]:
def preprocess_classification(df, test_size=0.2, random_state=123, use_smote=False):
    """
    Preprocesamiento:
     - Mapea variables binarias ('yes'/'no') y Gender.
     - Convierte CAEC a ordinal (no < Sometimes < Frequently < Always).
     - One-hot-encodes MTRANS.
     - Escala variables numéricas con StandardScaler.
     - Codifica target NObeyesdad con LabelEncoder (devuelve mapping).
     - Opcionalmente aplica SMOTE solo sobre el conjunto de entrenamiento.
    
    Devuelve:
     X_train_t, X_test_t, y_train_t, y_test_t, preprocess_pipeline, label_encoder, ordinal_encoders (dict)
    """
    df = df.copy()
    
    # --- Mapear binarias y género ---
    binary_cols = ['family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'CALC']
    
    for c in binary_cols:
        if c in df.columns:
            df[c] = df[c].map({'no': 0, 'yes': 1})
    
    # Gender
    if 'Gender' in df.columns:
        df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})
    
    # --- CAEC: ordinal ---
    if 'CAEC' in df.columns:
        caec_map = {'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
        # Hay posibilidad de valores atípicos; usamos get para evitar KeyError
        df['CAEC'] = df['CAEC'].map(lambda x: caec_map.get(x, np.nan))
    
    # --- Target encoding (label) ---
    if 'NObeyesdad' not in df.columns:
        raise ValueError("El DataFrame debe contener la columna 'NObeyesdad' como target.")
    le_target = LabelEncoder()
    df['target_encoded'] = le_target.fit_transform(df['NObeyesdad'])
    target_mapping = dict(zip(le_target.classes_, le_target.transform(le_target.classes_)))
    print("Mapping target (NObeyesdad) -> integer:")
    print(target_mapping)
    
    # Separar X e y
    X = df.drop(columns=['NObeyesdad', 'target_encoded'])
    y = df['target_encoded']
    
    # Identificar columnas numéricas y categóricas para el pipeline
    numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
    # Excluir columnas que acabamos de transformar ordinalmente (CAEC está en numeric_cols ahora, eso está bien)
    # Identificar columnas nominales para one-hot (MTRANS u otras object restantes)
    nominal_cols = X.select_dtypes(include=['object']).columns.tolist()
    
    # En este dataset sólo MTRANS suele ser nominal; pero manejamos generically
    onehot_cols = [c for c in nominal_cols if c in X.columns]
    
    # Construir ColumnTransformer
    transformers = []
    if numeric_cols:
        transformers.append(('num', StandardScaler(), numeric_cols))
    if onehot_cols:
        transformers.append(('ohe', OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore'), onehot_cols))
    
    preprocessor = ColumnTransformer(transformers=transformers, remainder='passthrough') 
    # remainder passthrough mantendrá columnas no listadas (por ejemplo si algo quedó fuera)
    
    # Pipeline (por si queremos añadir pasos de selección más adelante)
    pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
    
    # --- Train/test split (estratificado por target) ---
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y)
    
    # Ajustar y transformar con pipeline
    X_train_t = pipeline.fit_transform(X_train)
    X_test_t = pipeline.transform(X_test)
    
    # Convertir a DataFrame con nombres de columnas explicitas (opcional pero práctico)
    def _get_feature_names(column_transformer, input_features):
        # Similar a get_feature_names_out en sklearn>=1.0; compatible manualmente
        feature_names = []
        for name, trans, cols in column_transformer.transformers_:
            if name == 'remainder':
                if trans == 'passthrough':
                    feature_names.extend([c for c in input_features if c not in sum([list(cols) for _,_,cols in column_transformer.transformers_ if cols is not None and cols != 'remainder'], [])])
                continue
            if hasattr(trans, 'get_feature_names_out'):
                try:
                    names = list(trans.get_feature_names_out(cols))
                except Exception:
                    # Fallback
                    names = [f"{name}__{c}" for c in cols]
            else:
                if isinstance(trans, OneHotEncoder):
                    # crear nombres manualmente
                    cats = trans.categories_
                    # OneHotEncoder created categories_, but after fit — here it's already fit
                    for i, c in enumerate(cols):
                        cat_names = [f"{c}_{v}" for v in trans.categories_[i]]
                        # drop='first' => quitar el primero
                        if trans.drop == 'first':
                            cat_names = cat_names[1:]
                        feature_names.extend(cat_names)
                    continue
                else:
                    names = [f"{name}__{c}" for c in cols]
            feature_names.extend(names)
        return feature_names
    
    try:
        feature_names = _get_feature_names(pipeline.named_steps['preprocessor'], X_train.columns.tolist())
    except Exception:
        # Fallback simple
        feature_names = [f'feat_{i}' for i in range(X_train_t.shape[1])]
    
    X_train_t = pd.DataFrame(X_train_t, columns=feature_names, index=X_train.index)
    X_test_t = pd.DataFrame(X_test_t, columns=feature_names, index=X_test.index)
    
    print("\nFormas después del preprocesamiento:")
    print("X_train:", X_train_t.shape)
    print("X_test :", X_test_t.shape)
    print("y_train:", y_train.shape)
    print("y_test :", y_test.shape)
    
    # Mostrar distribución original y de entrenamiento
    print("\nDistribución original (total):")
    print(Counter(y))
    print("\nDistribución del train antes de resampling:")
    print(Counter(y_train))
    
    # --- Opcional: aplicar SMOTE sobre el conjunto de entrenamiento (solo si use_smote True) ---
    smote_applied = False
    if use_smote:
        try:
            from imblearn.over_sampling import SMOTE
            sm = SMOTE(random_state=random_state)
            X_resampled, y_resampled = sm.fit_resample(X_train_t, y_train)
            print("\nDespués de SMOTE (train):")
            print(Counter(y_resampled))
            X_train_t = pd.DataFrame(X_resampled, columns=X_train_t.columns)
            y_train = pd.Series(y_resampled)
            smote_applied = True
        except Exception as e:
            print("\nNo se pudo aplicar SMOTE. Para aplicarlo instala imbalanced-learn (pip install imbalanced-learn).")
            print("Error:", e)
            smote_applied = False
    
    return {
        'X_train': X_train_t,
        'X_test': X_test_t,
        'y_train': y_train,
        'y_test': y_test,
        'pipeline': pipeline,
        'label_encoder': le_target,
        'feature_names': feature_names,
        'smote_applied': smote_applied
    }

In [ ]:
Preprocesamiento  = preprocess_classification

### Mejor modelo para usar
- Gradient boosting (CatBoost / LightGBM / XGBoost) — mejor opción práctica.
- Random Forest — baseline robusto, fácil de interpretar con importancias.
- Multinomial Logistic Regression (con regularización) — buen baseline simple y explicable.
- Simple Neural Network (MLP) — si quieres comportamiento no lineal y combinaciones de features.
- Métodos sencillos (k-NN, Naive Bayes, SVM) — pueden servir como comparativas pero menos recomendados como modelos finales.

##### Por qué esos modelos:

- Datos mixtos (numéricos + categóricos + ordinal): los árboles y los boosters manejan bien interacciones, no requieren escalado estricto y toleran variables categóricas (CatBoost nativamente) — esto reduce riesgo de mal preprocesamiento.
- Multiclasificación con clases desbalanceadas: los boosters y RandomForest permiten pesos de clase o muestran buen rendimiento cuando se combinan  con rebalancing. CatBoost/LightGBM tienen parámetros para manejar imbalance y XGBoost tiene scale_pos_weight (aunque formato multiclass requiere atención).
- Tamaño del dataset (~2k filas): no es enorme. Los modelos muy complejos (redes profundas) pueden sobreajustar; boosting con regularización y early stopping suele generalizar mejor en pocos datos.